In [ ]:
from dataprep import prepare_data

DATA_DOWNLOAD_MODE = "INCREMENTAL" # FULL or INCREMENTAL
# when INCREMENTAL is chosen then it is assumed that historical data already exists and last 30 days data is fetched and merged to that


print("Start data prep")
# prepare_data(DATA_DOWNLOAD_MODE)

In [ ]:
import pandas as pd
full_df = pd.read_parquet('/home/abhay/work/pytasy/processed_output/delivery_parquet/')
people_df = pd.read_csv('downloads/people.csv')
# full_df.createOrReplaceTempView("all_matches")
full_df

In [ ]:
full_df['match_type'].unique()

In [ ]:
full_df['event_name'].unique()

In [ ]:
from country_lookup import venue_to_country, test_hosting_cities
import numpy as np

# Define the lookup function
def lookup_country(city, venue):
    if city is None:
        return venue_to_country.get(venue, None)
    return test_hosting_cities.get(city, None)  # Returns None if city not found
lookup_country_vec = np.vectorize(lookup_country)
# Apply the UDF
full_df['country'] = lookup_country_vec(full_df['city'], full_df['venue_name'])

full_df['year'] = pd.to_datetime(full_df["dt"]).dt.year.astype('str')
full_df['dt'] = full_df["dt"].astype('str')

In [ ]:
event_type='Indian Premier League'
ipl_stats = full_df[full_df['event_name'] == event_type]

In [ ]:
top_batters = ipl_stats.groupby(['batter_name']).agg(
    runs=('batter_runs', 'sum')
)
top_batters = top_batters.reset_index()
top_batters = top_batters[top_batters['runs'] > 100]
top_batters_names = set(top_batters['batter_name'])


In [ ]:
race_chart_stats = ipl_stats[ipl_stats['batter_name'].isin(top_batters_names)].groupby(['dt','batter_name']).agg(
    runs=('batter_runs', 'sum')
)
race_chart_stats = race_chart_stats.sort_values(by=['dt'])
race_chart_stats['runs'] = race_chart_stats.groupby(['batter_name'])['runs'].cumsum()
race_chart_stats = race_chart_stats.reset_index()
# flourish_stats = race_chart_stats
# race_chart_stats = race_chart_stats.pivot(index='dt', columns='batter_name', values='runs').fillna(method = 'ffill').fillna(0)
# race_chart_stats

In [ ]:
flourish_stats = race_chart_stats.pivot(index='batter_name', columns='dt', values='runs').fillna(method = 'ffill', axis=1).fillna(0)
flourish_stats = flourish_stats.reset_index()
flourish_stats.to_csv('/mnt/c/Users/91636/Documents/cricketbystats/ipl_top_scorers_race/flourish.csv', index=False)

In [ ]:
# import bar_chart_race as bcr
# # df = bcr.load_dataset('covid19_tutorial')
# bcr.bar_chart_race(
#     df=race_chart_stats,
#     filename='/mnt/c/Users/91636/Documents/cricketbystats/ipl_top_scorers_race/top_scorers.mp4',
#     orientation='h',
#     sort='desc',
#     n_bars=10,
#     fixed_order=False,
#     fixed_max=False,
#     steps_per_period=10,
#     interpolate_period=False,
#     label_bars=True,
#     bar_size=.95,
#     period_label={'x': .99, 'y': .25, 'ha': 'right', 'va': 'center'},
#     period_fmt='%B %d, %Y',
#     # period_summary_func=lambda v, r: {'x': .99, 'y': .18,
#     #                                   's': f'Total deaths: {v.nlargest(6).sum():,.0f}',
#     #                                   'ha': 'right', 'size': 8, 'family': 'Courier New'},
#     perpendicular_bar_func='median',
#     period_length=500,
#     figsize=(5, 3),
#     dpi=144,
#     cmap='dark12',
#     title='Top run getters in IPL',
#     title_size='',
#     bar_label_size=7,
#     tick_label_size=7,
#     # shared_fontdict={'family' : 'Helvetica', 'color' : '.1'},
#     scale='linear',
#     writer=None,
#     fig=None,
#     bar_kwargs={'alpha': .7},
#     filter_column_colors=False)  

In [ ]:

# player_id = 'b17e2f24' # KL rahul Pant
# event_type='Indian Premier League'
# home_country='India'
# people_df[people_df['unique_name'].str.contains('kl rahul', case=False)]
# ipl_stats = full_df[full_df['event_name'] == event_type]
# batter_stats = ipl_stats[ipl_stats['batter_id'] == player_id]
# batter_stats = batter_stats.sort_values(by=['dt', 'innings_number', 'over', 'ball'])
# innings_keys = ['dt', 'innings_number']

# batter_stats['batter_innings_runs'] = batter_stats.groupby(innings_keys)['batter_runs'].cumsum()
# batter_stats['batter_innings_deliveries'] = batter_stats.groupby(innings_keys).cumcount() + 1
# batter_stats['batter_innings_strike_rate'] = 100*batter_stats['batter_innings_runs'] / batter_stats['batter_innings_deliveries'].replace(0, 1)
# batter_stats
# batter_stats['innings_number'].unique()